# Hyperparameter Optimization

We have the same three steps as before, but we add an additional step for the HyperDrive configuration.

1. create a `*.py` file that'll be run by the remote compute
    - In this case, we've already created that here in gbt-train.py
1. Configure a ScriptRunConfig that will execute this script
1. **ADDED STEP** Wrap the ScriptRunConfig in a HyderDriveConfig
1. Submit the script for execution in AML Compute

In [ ]:
!python gbt-train.py --help

In [ ]:
from azureml.core import Workspace, Experiment, Run, Environment, Dataset, ScriptRunConfig
from azureml.train.hyperdrive import HyperDriveConfig, PrimaryMetricGoal, RandomParameterSampling, uniform, choice, normal, randint

# 2. Configure a `ScriptRun`

In [ ]:
USER_NAME = 'ENTER_YOUR_NAME_HERE'

ENVIRONMENT_NAME = f'sklearn-{USER_NAME}'
EXPERIMENT_NAME = f'diabetes-{USER_NAME}'
DATASET_NAME = f'diabetes-{USER_NAME}'
COMPUTE_TARGET_NAME = 'cpu-cluster'

DATA_PATH = "./data"
BLOB_PATH = f'/data/{USER_NAME}'

In [ ]:
from azureml.core import Workspace, ScriptRunConfig, Experiment, Dataset, Environment

ws = Workspace.from_config()
exp = Experiment(ws, EXPERIMENT_NAME)

diabetes_dataset = ws.datasets[DATASET_NAME]
sklearn_env = ws.environments[ENVIRONMENT_NAME]
cpu_cluster = ws.compute_targets[COMPUTE_TARGET_NAME]


In [ ]:
script_run = ScriptRunConfig(
    source_directory = '.',
    script='gbt-train.py',
    environment=sklearn_env,
    compute_target=cpu_cluster,
    arguments=['--data-path', diabetes_dataset.as_mount()]
)

# 3. Specify HyperDrive Run

In [ ]:
param_space = RandomParameterSampling({
    "learning-rate": normal(0.5, 0.05),
    "alpha": uniform(0.001, 0.1),
    "n-estimators": randint(400),
    "max-depth": randint(10)
})

hd_config = HyperDriveConfig(
    hyperparameter_sampling=param_space,
    primary_metric_name='rmse',
    primary_metric_goal=PrimaryMetricGoal.MINIMIZE,
    max_total_runs=15,
    max_concurrent_runs=2,
    run_config=script_run
)

In [ ]:
run = exp.submit(hd_config, tag={"Context": "Remote"})

In [ ]:
from azureml.widgets import RunDetails

RunDetails(run).show()